# Aprendizaje por Refuerzo para Generals.io

En este notebook, implementaremos un agente de aprendizaje por refuerzo utilizando Q-learning para jugar Generals.io. Utilizaremos el entorno de generals-bots y visualizaremos el progreso del entrenamiento.

**IMPORTANTE**: Antes de ejecutar este notebook, por favor:
1. Ve a Runtime -> Restart runtime
2. Espera a que el runtime se reinicie completamente
3. Luego ejecuta las celdas en orden

## 1. Instalación de Dependencias

Primero, necesitamos instalar las bibliotecas necesarias para nuestro proyecto.

In [ ]:
# Clonar el repositorio
!git clone https://github.com/strakam/generals-bots.git

# Instalar dependencias básicas
!pip install -q matplotlib pandas
!pip install -q pettingzoo

# Instalar generals-bots
%cd generals-bots
!pip install -q -e .
%cd ..

## 2. Importación de Bibliotecas

Importaremos todas las bibliotecas necesarias para nuestro proyecto.

In [ ]:
from generals.envs import PettingZooGenerals
from generals import GridFactory
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
import time

# Verificar la instalación
print("Versiones de las bibliotecas principales:")
print(f"NumPy: {np.__version__}")

## 3. Configuración del Entorno

Configuraremos el entorno de Generals.io con un grid personalizado y las propiedades deseadas.

In [ ]:
# Configuración del grid
grid_factory = GridFactory(
    mode="generalsio",  # Usamos el modo que replica el juego oficial
    min_grid_dims=(10, 10),
    max_grid_dims=(15, 15),
    mountain_density=0.2,
    city_density=0.05
)

# Crear el entorno
env = PettingZooGenerals(
    agents=["player1", "player2"],  # Lista de IDs de agentes
    grid_factory=grid_factory,
    truncation=1000,  # Número máximo de pasos por episodio
    speed_multiplier=1.0  # Velocidad del juego
)

# Verificar el espacio de observación y acción
print("Espacio de observación:", env.observation_space)
print("Espacio de acción:", env.action_space)

## 4. Implementación del Agente Q-Learning

Implementaremos una clase para nuestro agente Q-learning que aprenderá a jugar Generals.io.

In [ ]:
# Primero, veamos la estructura de una observación y el espacio de acción
observations, infos = env.reset()
print("Estructura de la observación:")
print(observations["player1"].keys())
print("\nDimensiones del grid:")
print(f"Filas: {observations['player1']['armies'].shape[0]}")
print(f"Columnas: {observations['player1']['armies'].shape[1]}")

class QLearningAgent:
    def __init__(self, learning_rate=0.1, discount_factor=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.q_table = {}
        # Obtener dimensiones del grid de la primera observación
        obs, _ = env.reset()
        self.grid_rows = obs["player1"]["armies"].shape[0]
        self.grid_cols = obs["player1"]["armies"].shape[1]
        # El número de acciones es el número de celdas en el grid
        self.num_actions = self.grid_rows * self.grid_cols
        print(f"Número de acciones posibles: {self.num_actions}")
        
    def get_state_key(self, obs):
        # Convertimos la observación en una clave única
        state_key = f"{obs['armies'].tobytes()}_{obs['generals'].tobytes()}_{obs['cities'].tobytes()}_{obs['mountains'].tobytes()}_{obs['owned_cells'].tobytes()}_{obs['opponent_cells'].tobytes()}"
        return state_key
    
    def is_valid_position(self, row, col):
        return 0 <= row < self.grid_rows and 0 <= col < self.grid_cols
    
    def get_action(self, obs):
        state_key = self.get_state_key(obs)
        if np.random.rand() <= self.epsilon:
            # Exploración: acción aleatoria válida
            # En generals-bots, todas las acciones son válidas si hay unidades en la celda
            valid_actions = np.zeros(self.num_actions)  # Inicializamos todas las acciones como inválidas
            
            # Marcamos como válidas solo las acciones que tienen unidades
            for row in range(self.grid_rows):
                for col in range(self.grid_cols):
                    if obs['armies'][row, col] > 0:
                        action_idx = row * self.grid_cols + col
                        valid_actions[action_idx] = 1
            
            valid_indices = np.where(valid_actions == 1)[0]
            if len(valid_indices) > 0:
                return np.random.choice(valid_indices)
            return 0  # Acción por defecto si no hay acciones válidas
        
        # Explotación: mejor acción según Q-table
        if state_key not in self.q_table:
            self.q_table[state_key] = np.zeros(self.num_actions)
            
        q_values = self.q_table[state_key].copy()
        # Invalidamos acciones que no tienen unidades
        for row in range(self.grid_rows):
            for col in range(self.grid_cols):
                action_idx = row * self.grid_cols + col
                if obs['armies'][row, col] == 0:
                    q_values[action_idx] = float('-inf')
        
        return np.argmax(q_values)
    
    def update(self, obs, action, reward, next_obs, done):
        state_key = self.get_state_key(obs)
        next_state_key = self.get_state_key(next_obs)
        
        if state_key not in self.q_table:
            self.q_table[state_key] = np.zeros(self.num_actions)
        if next_state_key not in self.q_table:
            self.q_table[next_state_key] = np.zeros(self.num_actions)
            
        current_q = self.q_table[state_key][action]
        next_max_q = np.max(self.q_table[next_state_key])
        
        new_q = current_q + self.learning_rate * (reward + self.discount_factor * next_max_q * (1 - done) - current_q)
        self.q_table[state_key][action] = new_q
        
        if done:
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

## 5. Entrenamiento del Agente

Implementaremos la función de entrenamiento para nuestro agente.

In [ ]:
def train_agent(episodes=1000, max_steps=1000):
    agent = QLearningAgent()
    rewards_history = []
    wins_history = []
    
    for episode in range(episodes):
        observations, infos = env.reset()
        total_reward = 0
        won = False
        
        for step in range(max_steps):
            # Turno del agente
            action = agent.get_action(observations["player1"])
            next_observations, rewards, terminations, truncations, infos = env.step({"player1": action, "player2": 0})
            
            agent.update(observations["player1"], action, rewards["player1"], next_observations["player1"], terminations["player1"])
            total_reward += rewards["player1"]
            
            if infos["player1"].get("won", False):
                won = True
                
            if terminations["player1"] or truncations["player1"]:
                break
                
            observations = next_observations
            
        rewards_history.append(total_reward)
        wins_history.append(won)
        
        if episode % 10 == 0:
            print(f"Episodio: {episode}, Recompensa Total: {total_reward:.2f}, Épsilon: {agent.epsilon:.2f}")
            
    return rewards_history, wins_history

## 6. Visualización de Resultados

Implementaremos funciones para visualizar el progreso del entrenamiento.

In [ ]:
def plot_training_results(rewards_history, wins_history):
    plt.figure(figsize=(15, 5))
    
    # Gráfico de recompensas
    plt.subplot(1, 2, 1)
    plt.plot(rewards_history)
    plt.title('Historial de Recompensas')
    plt.xlabel('Episodio')
    plt.ylabel('Recompensa Total')
    
    # Gráfico de victorias
    plt.subplot(1, 2, 2)
    plt.plot(pd.Series(wins_history).rolling(10).mean())
    plt.title('Tasa de Victoria (Media Móvil)')
    plt.xlabel('Episodio')
    plt.ylabel('Tasa de Victoria')
    
    plt.tight_layout()
    plt.show()

## 7. Ejecución del Entrenamiento

Ahora ejecutaremos el entrenamiento y visualizaremos los resultados.

In [ ]:
# Ejecutar el entrenamiento
rewards_history, wins_history = train_agent(episodes=1000)

# Visualizar resultados
plot_training_results(rewards_history, wins_history)

## 8. Evaluación del Agente Entrenado

Finalmente, evaluaremos el rendimiento de nuestro agente entrenado en algunos juegos de prueba.

In [ ]:
def evaluate_agent(episodes=10):
    agent = QLearningAgent()
    agent.epsilon = 0  # Desactivamos la exploración para la evaluación
    
    wins = 0
    total_rewards = []
    
    for episode in range(episodes):
        observations, infos = env.reset()
        total_reward = 0
        done = False
        
        while not done:
            action = agent.get_action(observations["player1"])
            next_observations, rewards, terminations, truncations, infos = env.step({"player1": action, "player2": 0})
            
            total_reward += rewards["player1"]
            observations = next_observations
            done = terminations["player1"] or truncations["player1"]
            
            if infos["player1"].get("won", False):
                wins += 1
                
        total_rewards.append(total_reward)
        print(f"Episodio de evaluación {episode + 1}: Recompensa = {total_reward:.2f}")
        
    print(f"\nTasa de victoria: {wins/episodes*100:.2f}%")
    print(f"Recompensa promedio: {np.mean(total_rewards):.2f}")
    
    return wins/episodes, np.mean(total_rewards)

In [ ]:
# Evaluar el agente
win_rate, avg_reward = evaluate_agent(episodes=10)